In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base

engine = create_engine('sqlite:///db/projektai.db')
Base = declarative_base()
session = sessionmaker(engine)()

In [3]:
from datetime import datetime
from sqlalchemy import Column, Integer, String, Float, DateTime

In [4]:
from sqlalchemy.orm import relationship
from sqlalchemy import ForeignKey

## Projektai - single table DB

In [ ]:
class Projektas(Base):
    __tablename__ = "Porojektas"
    id = Column(Integer, primary_key=True)
    pavadinimas = Column("pavadinimas", String(50))
    kaina = Column("kaina", Float())
    sukurta = Column("sukurta", DateTime, default=datetime.utcnow)

    def __init__(self, pavadinimas, kaina):
        self.pavadinimas = pavadinimas
        self.kaina = kaina

    def __repr__(self):
        return f"({self.id}, {self.pavadinimas}, {self.kaina}, {self.sukurta})"


Base.metadata.create_all(engine)

### INSERT

In [ ]:
gka = Projektas("Gero Kodo Akademija", 0)
session.add(gka)
session.commit()

In [ ]:
sandeliukas = Projektas("Sandeliukas 3.0", 17300)
session.add(sandeliukas)
session.commit()

In [ ]:
nepaejes = Projektas("Kreiva Programulka", 700)
session.add(nepaejes)
session.commit()

In [ ]:
print(gka)
print(sandeliukas)

### SELECT'ai

In [ ]:
projektas = session.get(Projektas, 2)
print(projektas)

In [ ]:
projektai = session.query(Projektas).all()
print(projektai)

In [ ]:
nemokami = session.query(Projektas).filter(Projektas.kaina == 0).all()
print(nemokami)

### UPDATE

In [ ]:
gka.kaina = 50000
session.add(gka)
session.commit()

### DELETE

In [ ]:
kreiva = session.get(Projektas, 3)
session.delete(kreiva)
session.commit()

## Relationships (tevai/vaikai)

In [ ]:
m21_engine = create_engine('sqlite:///db/m21_tevai_vaikai.db')
m21_session = sessionmaker(m21_engine)()
BaseM21 = declarative_base()


class Vaikas(BaseM21):
    __tablename__ = "vaikas"
    id = Column(Integer, primary_key=True)
    vardas = Column("vardas", String(50))
    pavarde = Column("pavarde", String(50))
    mokykla = Column("mokykla", String(50))
    tevai = relationship("Tevas", back_populates='vaikas')

    def __repr__(self):
        return f"({self.vardas}, {self.pavarde}, {self.mokykla})"


class Tevas(BaseM21):
    __tablename__ = "tevas"
    id = Column(Integer, primary_key=True)
    vardas = Column("vardas", String(50))
    pavarde = Column("pavarde", String(50))
    vaikas_id = Column(Integer, ForeignKey('vaikas.id'))
    vaikas = relationship("Vaikas", back_populates='tevai')

    def __repr__(self):
        return f"({self.vardas}, {self.pavarde})"


BaseM21.metadata.create_all(m21_engine)

In [ ]:
emilija = Vaikas(vardas="Emilija", pavarde="J", mokykla="Pasaulis")
mama = Tevas(vardas="Nico", pavarde="P", vaikas=emilija)
tevas = Tevas(vardas="Kestas", pavarde="J", vaikas=emilija)
m21_session.add_all([emilija, mama, tevas])
m21_session.commit()

In [ ]:
kestas = m21_session.get(Tevas, 2)
print(kestas)

In [ ]:
tevai = m21_session.query(Tevas).filter(Tevas.vaikas == emilija).all()
print(tevai)

In [ ]:
tevai = m21_session.query(Tevas).filter_by(vaikas=emilija).all()
print(tevai)

In [ ]:
naujas = Vaikas(vardas="Naujas", pavarde="G")
trecias = Tevas(vardas="Geras", pavarde="G", vaikas=naujas)
m21_session.add_all([naujas, trecias])
m21_session.commit()

In [ ]:
visi_tevai = m21_session.query(Tevas).all()
visi_vaikai = m21_session.query(Vaikas).all()
print(visi_tevai, visi_vaikai)

In [ ]:
visi_tevai[2].vaikas = emilija
m21_session.add(visi_tevai[2])
m21_session.commit()

In [ ]:
ketvirtas = Tevas(vardas="Super", pavarde="Mama")
naujas.tevai.append(ketvirtas)
m21_session.add_all([ketvirtas, naujas])
m21_session.commit()

In [ ]:
atnaujintas = m21_session.query(Vaikas).filter_by(vardas="Naujas").all()
print(atnaujintas)
print(atnaujintas[0].tevai)

In [ ]:
supermama = m21_session.query(Tevas).filter_by(pavarde="Mama").one()
print(supermama.vaikas)

## Many 2 Many tevai

In [5]:
from sqlalchemy import Table

m2m_engine = create_engine('sqlite:///db/m2m_tevai_vaikai.db')
m2m_session = sessionmaker(m2m_engine)()
BaseM2M = declarative_base()


In [6]:
tevai_vaikai = Table(
    'tevai_vaikai', 
    BaseM2M.metadata,
    Column('tevas_id', Integer, ForeignKey('vaikas.id')),
    Column('vaikas_id', Integer, ForeignKey('tevas.id')), 
)

In [ ]:
# class TevaiVaikai(BaseM2M):
#     __tablename__ = 'tevai_vaikai'
#     id = Column(Integer, primary_key=True)
#     tevas_id = Column('tevas_id', Integer, ForeignKey('tevas.id'))
#     vaikas_id = Column('vaikas_id', Integer, ForeignKey('vaikas.id'))


In [7]:
class Tevas(BaseM2M):
    __tablename__ = 'tevas'
    id = Column(Integer, primary_key=True)
    vardas = Column("vardas", String(50))
    pavarde = Column("pavarde", String(50))
    vaikai = relationship("Vaikas", back_populates="tevai", secondary=tevai_vaikai)

    def __repr__(self):
        return f"({self.vardas}, {self.pavarde})"


class Vaikas(BaseM2M):
    __tablename__ = 'vaikas'
    id = Column(Integer, primary_key=True)
    vardas = Column("vardas", String(50))
    pavarde = Column("pavarde", String(50))
    mokykla = Column("mokykla", String(50))
    tevai = relationship("Tevas", back_populates="vaikai", secondary=tevai_vaikai)

    def __repr__(self):
        return f"({self.vardas}, {self.pavarde}, {self.mokykla})"


BaseM2M.metadata.create_all(m2m_engine)

In [9]:
tevas = Tevas(vardas="Kestutis", pavarde="Januskevicius")
mama = Tevas(vardas="Nicoletta", pavarde="Januskeviciene")
emilija = Vaikas(vardas="Emilija", pavarde="Januskeviciute")
maija = Vaikas(vardas="Maija", pavarde="Januskeviciute")
marco = Vaikas(vardas="Marco", pavarde="Januskevicius")

tevas.vaikai.append(emilija)
tevas.vaikai.append(maija)
tevas.vaikai.append(marco)
mama.vaikai = [emilija, maija, marco]

m2m_session.add_all([tevas, mama])
m2m_session.commit()


In [13]:
nico = m2m_session.query(Tevas).filter(Tevas.vardas.ilike("nico%")).one()
print(nico)
print(nico.vaikai)

(Nicoletta, Januskeviciene)
[(Emilija, Januskeviciute, None), (Maija, Januskeviciute, None), (Marco, Januskevicius, None)]


In [14]:
for vaikas in nico.vaikai:
    print(vaikas)
    print(vaikas.tevai)

(Emilija, Januskeviciute, None)
[(Kestutis, Januskevicius), (Nicoletta, Januskeviciene)]
(Maija, Januskeviciute, None)
[(Kestutis, Januskevicius), (Nicoletta, Januskeviciene)]
(Marco, Januskevicius, None)
[(Kestutis, Januskevicius), (Nicoletta, Januskeviciene)]


In [19]:
kestas = nico.vaikai[0].tevai[0]
print(kestas)

(Kestutis, Januskevicius)


In [20]:
emilija = kestas.vaikai[0]
print(emilija)

(Emilija, Januskeviciute, None)


In [21]:
emilija.mokykla = "Pasaulis"
kestas.vaikai.remove(emilija)
m2m_session.add_all([emilija, kestas])
m2m_session.commit()

In [24]:
nepriklausoma = m2m_session.query(Vaikas).filter(Vaikas.vardas=="Emilija").one()
print(nepriklausoma)
print(nepriklausoma.tevai)

(Emilija, Januskeviciute, Pasaulis)
[(Nicoletta, Januskeviciene)]


In [26]:
tevas = m2m_session.query(Tevas).filter_by(vardas="Kestutis").one()
print(tevas, tevas.vaikai)

(Kestutis, Januskevicius) [(Maija, Januskeviciute, None), (Marco, Januskevicius, None)]


In [27]:
print(nepriklausoma.tevai[0].vaikai)

[(Emilija, Januskeviciute, Pasaulis), (Maija, Januskeviciute, None), (Marco, Januskevicius, None)]
